In [1]:
# !pip install --upgrade numpy

In [23]:
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn import model_selection 
from sklearn.manifold import TSNE

In [3]:
# Keras

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding


Using TensorFlow backend.


In [81]:

## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)


In [5]:
df = pd.read_csv('Consumer_Complaints.csv')

C:\Users\Tami\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning:

Columns (4,5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.



In [6]:
from io import StringIO

col = ['Product', 'Consumer complaint narrative']
df = df[col]
df = df[pd.notnull(df['Consumer complaint narrative'])]
df.columns = ['Product', 'Consumer_complaint_narrative']
df['category_id'] = df['Product'].factorize()[0]

df.head()

,Product,Consumer_complaint_narrative,category_id
225,Debt collection,Im in complete shock about this negative item ...,0
249,Debt collection,This so called receivable performance Fraudule...,0
287,"Credit reporting, credit repair services, or o...",Creditor Information MONTEREY COLL SVC XXXX XX...,1
331,Debt collection,Elite Financial has contacted myself and my fi...,0
534,"Credit reporting, credit repair services, or o...",I had sought to have some erroneous reporting ...,1


In [7]:
category_id_df = df[['Product', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Product']].values)

category_id_df

,Product,category_id
225,Debt collection,0
287,"Credit reporting, credit repair services, or o...",1
875,Mortgage,2
1007,Credit card or prepaid card,3
1361,Checking or savings account,4
2441,"Payday loan, title loan, or personal loan",5
2446,"Money transfer, virtual currency, or money ser...",6
3417,Student loan,7
3724,Vehicle loan or lease,8
413876,Credit reporting,9


In [17]:

df


,Product,Consumer_complaint_narrative,category_id
225,Debt collection,Im in complete shock about this negative item ...,0
249,Debt collection,This so called receivable performance Fraudule...,0
287,"Credit reporting, credit repair services, or o...",Creditor Information MONTEREY COLL SVC XXXX XX...,1
331,Debt collection,Elite Financial has contacted myself and my fi...,0
534,"Credit reporting, credit repair services, or o...",I had sought to have some erroneous reporting ...,1
538,"Credit reporting, credit repair services, or o...",Ever since XXXX had a breach of their system t...,1
665,"Credit reporting, credit repair services, or o...",On XX/XX/2018 an inquiry was made by XXXX cred...,1
781,Debt collection,XX/XX/2018 I obtained XXXX as my mobile servic...,0
802,"Credit reporting, credit repair services, or o...",Several companies listed inquires on my report...,1
875,Mortgage,My loan was purchased by Freedom Mortgage in X...,2


In [71]:
labels =df['category_id']
len(labels)

350651

In [93]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(df['Consumer_complaint_narrative'], df['category_id'])

In [99]:
len(test_y)+len(train_x)

350651

In [74]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    return text

In [20]:
import nltk

nltk.download('stopwords')

df['Consumer_complaint_narrative'] = df['Consumer_complaint_narrative'].map(lambda x: clean_text(x))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tami\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [75]:

### Create sequence
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['Consumer_complaint_narrative'])#train_x[0:])
sequences = tokenizer.texts_to_sequences(df['Consumer_complaint_narrative'])#(train_x[0:])
data = pad_sequences(sequences, maxlen=50)

In [92]:
len(valid_y)

87663

In [100]:
## Network architecture
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# ## Fit the model
model.fit(data, labels, validation_split=0.3, epochs=3)

ValueError: Error when checking target: expected dense_12 to have shape (18,) but got array with shape (1,)

In [70]:
# model.predict(data)

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [88]:
results = model.evaluate(data, labels)

print(results[1])

350651/350651 [==============================] - 92s 261us/step
-44.57471954242593


In [89]:
print(results[1])

0.22407750156138154
